In [44]:
%config IPCompleter.greedy=True

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [46]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) #(Input Channels, Output Channel, Filter size)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120) #(Input, Output)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_features(self, x):
        size = x.size()[1:]
        out = 1
        for s in size:
            out*=s
        return out 
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [47]:
print(list(net.parameters())[0].size())

torch.Size([6, 1, 5, 5])


In [48]:
random = torch.rand(1, 1, 32, 32)
random

tensor([[[[0.2325, 0.0474, 0.6000,  ..., 0.3069, 0.9937, 0.7022],
          [0.8261, 0.7403, 0.3482,  ..., 0.5670, 0.8912, 0.6368],
          [0.8348, 0.7942, 0.0749,  ..., 0.2708, 0.0115, 0.8125],
          ...,
          [0.8237, 0.7330, 0.1904,  ..., 0.4666, 0.2018, 0.1995],
          [0.6261, 0.7165, 0.5483,  ..., 0.0370, 0.0478, 0.4137],
          [0.6693, 0.5046, 0.0920,  ..., 0.3681, 0.9990, 0.1125]]]])

In [49]:
out = net(random)
print(out)

tensor([[-0.0984, -0.1055,  0.0859, -0.0452,  0.0598, -0.1328, -0.0876, -0.0992,
         -0.0424,  0.0508]], grad_fn=<AddmmBackward>)


### Loss 

In [50]:
target = torch.randn(1, 10)
target.size()

torch.Size([1, 10])

In [51]:
critereon = nn.MSELoss()
loss = critereon(out, target)
print(loss)

tensor(1.1146, grad_fn=<MseLossBackward>)


In [52]:
loss.grad_fn

### Backprop

In [53]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward()
print(net.conv1.bias.grad)

None
tensor([ 0.0009, -0.0008,  0.0048, -0.0018, -0.0115,  0.0069])


### Updating weights 

Manually


In [54]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*lr)

Using package

In [56]:
import torch.optim as optim

In [57]:
optimizer = optim.SGD(net.parameters(), lr = 0.01)

In [58]:
optimizer.zero_grad()
output = net(random)
loss = critereon(output, target)
loss.backward()
optimizer.step()